**------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
pip install rouge

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=203cb74bce13a21e6628bcd610923ec3a362185f09a8d0d1166df4e9c00c13c2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
import tensorflow as tf

# Step 1: Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Step 2: Mount Google Drive
drive.mount('/content/drive')

# Step 3: Load datasets
train_data = pd.read_csv("/content/drive/MyDrive/CNN dataset/train.csv").drop('id', axis=1)
test_data = pd.read_csv("/content/drive/MyDrive/CNN dataset/test.csv").drop('id', axis=1)
validation_data = pd.read_csv("/content/drive/MyDrive/CNN dataset/validation.csv").drop('id', axis=1)

# Sample data for faster testing
train_sample = train_data.sample(frac=0.001, random_state=42)
test_sample = test_data.sample(frac=0.01, random_state=42)
validation_sample = validation_data.sample(frac=0.01, random_state=42)

# Extract articles and highlights
train_articles, train_highlights = train_sample['article'], train_sample['highlights']
test_articles, test_highlights = test_sample['article'], test_sample['highlights']
validation_articles, validation_highlights = validation_sample['article'], validation_sample['highlights']

# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Step 4: Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the datasets
train_articles = train_articles.apply(preprocess_text)
train_highlights = train_highlights.apply(preprocess_text)
test_articles = test_articles.apply(preprocess_text)
test_highlights = test_highlights.apply(preprocess_text)
validation_articles = validation_articles.apply(preprocess_text)
validation_highlights = validation_highlights.apply(preprocess_text)

# Step 5: Tokenization and Padding
tokenizer = Tokenizer(num_words=3906)  # Use your vocab size
tokenizer.fit_on_texts(train_articles)

# Convert text to sequences of indices
train_articles_indices = tokenizer.texts_to_sequences(train_articles)
train_highlights_indices = tokenizer.texts_to_sequences(train_highlights)

# Pad sequences to the same length
MAX_SEQUENCE_LENGTH = 50
train_articles_padded = pad_sequences(train_articles_indices, maxlen=MAX_SEQUENCE_LENGTH)
train_highlights_padded = pad_sequences(train_highlights_indices, maxlen=MAX_SEQUENCE_LENGTH)

# Step 6: Model definition
model = Sequential([
    Embedding(input_dim=3906, output_dim=256, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    Dense(3906, activation='softmax')  # Output layer with vocabulary size
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 7: Train the model
history = model.fit(
    train_articles_padded,
    np.expand_dims(train_highlights_padded, axis=-1),  # Expand dims to add channel dimension
    epochs=10,
    batch_size=64
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
